In [2]:
using Revise

In [12]:
@assert Threads.nthreads() > 1

In [13]:
using MeshCatMechanisms
using MeshCat
using RigidBodyDynamics
using LCPSim
using AtlasRobot
using Blink
using RigidBodyDynamics
using RigidBodyDynamics.Contact
using Rotations
using LearningMPC
using LearningMPC.Models
using Gurobi
using StaticArrays
using RigidBodySim
using OSQP.MathOptInterfaceOSQP
using QPControl
using MathOptInterface
const MOI = MathOptInterface

MathOptInterface

In [14]:
atlas_sim = LearningMPC.Models.PlanarAtlas(:simulation);

mvis = MechanismVisualizer(atlas_sim)
open(mvis, Window())
set_configuration!(mvis, configuration(nominal_state(atlas_sim)))

AtlasRobot.packagepath() = "/home/rdeits/locomotion/explorations/learning-mpc-2/packages/v0.6/AtlasRobot/deps"


In [15]:


function compose(controllers...)
    function (τ, t, state)
        for c in controllers
            c(τ, t, state)
        end
        τ
    end
end
   
zero_controller(τ::AbstractVector, t::Number, state::MechanismState) = τ .= 0

zero_controller (generic function with 1 method)

In [111]:
function defaultoptimizer()
    optimizer = OSQPOptimizer()
    MOI.set!(optimizer, OSQPSettings.Verbose(), false)
    MOI.set!(optimizer, OSQPSettings.EpsAbs(), 1e-5)
    MOI.set!(optimizer, OSQPSettings.EpsRel(), 1e-5)
    MOI.set!(optimizer, OSQPSettings.MaxIter(), 5000)
    MOI.set!(optimizer, OSQPSettings.AdaptiveRhoInterval(), 25) # required for deterministic behavior
#     MOI.set!(optimizer, OSQPSettings.WarmStart(), false)
    optimizer
end

defaultoptimizer (generic function with 1 method)

In [215]:
# atlas_control = LearningMPC.Models.PlanarAtlas(:control);
feet = Dict(:left => findbody(mechanism(atlas_sim), "l_foot_sole"), :right => findbody(mechanism(atlas_sim), "r_foot_sole"))
hands = Dict(:left => findbody(mechanism(atlas_sim), "l_hand_mount"), :right => findbody(mechanism(atlas_sim), "r_hand_mount"));
floor = HalfSpace3D(Point3D(root_frame(mechanism(atlas_sim)), 0., 0, 0), FreeVector3D(root_frame(mechanism(atlas_sim)), 0., 0, 1))

xstar = nominal_state(atlas_sim)
Δt = 0.05
nq = num_positions(xstar)
nv = num_velocities(xstar)
qq = fill(0.1, nq)
qq[1] = 10
qq[2] = 100
qq[3] = 500
# qq[configuration_range(xstar, findjoint(mechanism(atlas_sim), "r_leg_hpx"))] .= 100
# qq[configuration_range(xstar, findjoint(mechanism(atlas_sim), "l_leg_hpx"))] .= 100
qv = fill(0.1, nv)
qv[velocity_range(xstar, findjoint(xstar.mechanism, "floating_base"))] .= [100, 20, 50]
Q = diagm(vcat(qq, qv))
R = diagm(fill(1e-6, nv))
lqrsol = LearningMPC.LQRSolution(xstar, Q, R, Δt, 
    [Point3D(default_frame(body), 0., 0, 0) for body in 
            findbody.(mechanism(atlas_sim), ["r_foot_sole", "l_foot_sole"])])


(::LQRSolution) (generic function with 2 methods)

In [216]:
effort_limiter = let effort_bounds = LCPSim.all_effort_bounds(mechanism(atlas_sim))
    function (τ::AbstractVector, t::Number, state::MechanismState)
        τ .= clamp.(τ, effort_bounds)
        τ
    end
end

bounds_enforcer = let position_bounds = LCPSim.all_configuration_bounds(mechanism(atlas_sim))
    function (τ::AbstractVector, t::Number, state::MechanismState)
        # TODO: handle q̇ vs v correctly
        for i in 1:num_positions(state)
            kp = 1000
            kd = 0.1 * kp
            if configuration(state)[i] > position_bounds[i].upper
                violation = configuration(state)[i] - position_bounds[i].upper
                τ[i] -= kp * violation
                τ[i] -= kd * velocity(state)[i]
            elseif configuration(state)[i] < position_bounds[i].lower
                violation = position_bounds[i].lower - configuration(state)[i]
                τ[i] += kp * violation 
                τ[i] -= kd * velocity(state)[i]
            end
        end
        τ
    end
end

damper = function(τ, t, state)
    τ .= τ .- 1.0 .* velocity(state)
end

(::#318) (generic function with 1 method)

In [232]:
function build_contact_controller(robot, contact_state, lqrsol)
    lowlevel = QPControl.MPCController{4}(mechanism(robot), defaultoptimizer())
    stage = QPControl.addstage!(lowlevel, lqrsol.Δt)
    for (i, (side, body)) in enumerate(feet)
        contact_point = QPControl.addcontact!(stage,
            body,
            Point3D(default_frame(body), 0., 0, 0),
            FreeVector3D(default_frame(body), 0., 0, 1),
            1.0,
            floor
        )
        if contact_state[i]
            contact_point.maxnormalforce = 1e9
        else
            contact_point.maxnormalforce = 0
        end
    end

    lowlevel.running_state_cost.Q .= lqrsol.Q
    lowlevel.running_state_cost.q .= 0
    lowlevel.running_state_cost.x0 .= lqrsol.x0
    lowlevel.running_input_cost.Q .= lqrsol.R
    lowlevel.running_input_cost.q .= 0
    lowlevel.running_input_cost.x0 .= 0
    lowlevel.terminal_state_cost.Q .= lqrsol.S
    lowlevel.terminal_state_cost.q .= 0
    lowlevel.terminal_state_cost.x0 .= lqrsol.x0;
    lowlevel
end
    

function build_mixed_controller(robot, lqrsol)
    contact_states = collect(IterTools.product([true, false], [true, false]))
#     contact_states = [[true, true]]
    controllers = map(contact_states) do contact_state
        @show contact_state
        build_contact_controller(robot, contact_state, lqrsol)
    end
    τs = [zeros(num_velocities(mechanism(robot))) for controller in controllers]
    objective_values = zeros(length(controllers))
    
    function control(τ, t, x)
        let controllers = controllers,
            τs = τs,
            objective_values = objective_values
            
#             Threads.@threads 
            for i in eachindex(controllers)
                controller = controllers[i]
                if !controller.initialized
                    QPControl.initialize!(controller)
                end
                for stage in QPControl.stages(controller)
                    QPControl.copyto!(stage.state, x)
                end
                model = controller.qpmodel
                SimpleQP.solve!(model)
                τs[i] .= SimpleQP.value.(model, controller.stages[1].input)
#                 @show SimpleQP.primalstatus(model)
                if SimpleQP.primalstatus(model) == MOI.FeasiblePoint
                    objective_values[i] = SimpleQP.objectivevalue(model)
                else
                    objective_values[i] = Inf
                end
#                 for i in 1:2
#                     @show SimpleQP.value.(controller.qpmodel, 
#                         collect(values(controller.stages[1].contacts))[i][1][1].force_local.v)
#                 end
            end
        end
        best_cost = Inf
        best_model = 0
        for i in eachindex(controllers)
            if objective_values[i] < best_cost
                best_model = i
                best_cost = objective_values[i]
                τ .= τs[i]
            end
        end
        if best_cost == Inf
            @show configuration(state) velocity(state)
            error("infeasible")
        end
#         @show best_model contact_states[best_model]
        τ
    end, controllers
end
    
mixed_control, lowlevel_controllers = build_mixed_controller(atlas_sim, lqrsol);

contact_state = (true, true)
contact_state = (false, true)
contact_state = (true, false)
contact_state = (false, false)


In [233]:
state = nominal_state(atlas_sim)
set_velocity!(state, findjoint(mechanism(atlas_sim), "floating_base"), [-0.2, 0, 0])
final_time = 5.0

composed = PeriodicController(similar(velocity(state)), 0.01, compose(mixed_control, effort_limiter))
controlcallback = DiffEqCallbacks.PeriodicCallback(composed)
composed = compose(composed, bounds_enforcer, damper)

dynamics = Dynamics(mechanism(atlas_sim), composed)
problem = ODEProblem(dynamics, state, (0., final_time), callback = CallbackSet(controlcallback)) # CallbackSet(mvis, state, max_fps = 30.)))

@time sol = RigidBodySim.solve(problem, Tsit5(), abs_tol = 1e-8, dt = 1e-6);
# @time sol = RigidBodySim.solve(problem, Tsit5(), abs_tol = 1e-8, dt = 1e-6);

 17.563385 seconds (1.90 M allocations: 180.969 MiB, 2.91% gc time)


In [234]:
RigidBodySim.animate(mvis, mvis.state, sol)